# Multi Agentes com Ollama + Phi-4
--------------------------
Este notebook demonstra como integrar o modelo Phi-4 via Ollama e orquestrar
tarefas com CrewAI e LangChain.

Funcionalidades principais:
    • Configuração de agentes autônomos usando CrewAI.
    • Criação e execução de tarefas com LangChain + Ollama.
    • Exemplo de pipeline para interação e geração de relatórios.

Dependências principais:
    pip install crewai langchain langchain_ollama python-dotenv

Requisitos:
    • Python 3.10+ (recomendado Anaconda/conda)
    • Servidor Ollama em execução com o modelo phi4 disponível.

Como usar:
    1. Ajuste as variáveis de ambiente no arquivo .env (se necessário).
    2. Execute as células na ordem: imports → configuração → criação dos agentes.
    3. Edite os parâmetros (cidades, datas, interesses etc.) para seus testes.

Observações:
    - Verifique se o modelo `phi4` já foi baixado no Ollama (`ollama pull phi4`).
    - Revise e adapte os prompts de acordo com o fluxo desejado.
"""


In [1]:
from crewai.tools import tool

from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchResults



class SearchTools:
    @tool("Pesquisa na internet")
    def search_tavily(query: str = "") -> str:
        """
        Pesquisa na web usando a API Tavily.
        Recomendado para informações estruturadas e recentes.
        """
        search_tavily = TavilySearch(max_results=4)
        search_res = search_tavily.invoke(query)
        return search_res

    @tool("Pesquisa na internet com DuckDuckGo")
    def search_duckduckgo(query: str):
        """
        Pesquisa na web usando DuckDuckGo.
        Retorna uma lista de resultados.
        """
        search_tool = DuckDuckGoSearchResults(num_results=4, verbose=True)
        return search_tool.run(query)


In [2]:
class CalculatorTools:
    @tool("Faça um cálculo")
    def calculate(operation):
        """Useful to perform any mathematical calculations,
        like sum, minus, multiplication, division, etc.
        The input to this tool should be a mathematical
        expression, a couple examples are `200*7` or `5000/2*10`
        """
        try:
            return eval(operation)
        except SyntaxError:
            return "Erro: Sintaxe inválida"

CalculatorTools.calculate

Tool(name='Faça um cálculo', description='Tool Name: Faça um cálculo\nTool Arguments: {}\nTool Description: Useful to perform any mathematical calculations,\n        like sum, minus, multiplication, division, etc.\n        The input to this tool should be a mathematical\n        expression, a couple examples are `200*7` or `5000/2*10`\n        ', env_vars=[], args_schema=<class 'abc.Façaumcálculo'>, description_updated=False, cache_function=<function BaseTool.<lambda> at 0x000001FC52888160>, result_as_answer=False, max_usage_count=None, current_usage_count=0, func=<function CalculatorTools.calculate at 0x000001FC773B5EA0>)

In [3]:
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from typing import List, Dict, Any
import os
# Optional: environment variables via .env
try:
    from dotenv import load_dotenv
    load_dotenv()
except Exception:
    pass
# =========================
# Config & Helpers
# =========================
# Aceita ambos nomes de variáveis para compatibilidade:
# - OLLAMA_ENDPOINT ou OLLAMA_BASE_URL
# - OLLAMA_MODEL ou LLM_MODEL_NAME
OLLAMA_BASE_URL = os.getenv("OLLAMA_ENDPOINT", os.getenv("OLLAMA_BASE_URL", "http://localhost:11434"))
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", os.getenv("LLM_MODEL_NAME", "phi4"))
LLM_TEMPERATURE = float(os.getenv("LLM_TEMPERATURE", "0.1"))

# Para CrewAI via LiteLLM, o nome do modelo deve conter o provedor (ex.: "ollama/phi4")
PROVIDER_PREFIX = os.getenv("PROVIDER_PREFIX", "ollama/")
CREW_MODEL = f"{PROVIDER_PREFIX}{OLLAMA_MODEL}"  # "ollama/phi4"

# =========================
# LLM p/ Chat direto (LangChain)
# =========================
_llm_instance = None

def load_llm() -> ChatOllama:
    """Instância única para o utilitário 'model_response' (chat direto)."""
    global _llm_instance
    if _llm_instance is None:
        _llm_instance = ChatOllama(
            model=OLLAMA_MODEL,               # "phi4"
            base_url=OLLAMA_BASE_URL,         # "http://localhost:11434"
            temperature=LLM_TEMPERATURE,
        )
    return _llm_instance

def model_response(system_prompt: str, user_query: str,
                   chat_history: List[List[str]] | None = None) -> str:
    llm = load_llm()
    # Inclui a instrução de idioma no início do system prompt
    messages = [("system", f"{system_prompt}\nResponda sempre em português do Brasil.")]
    if chat_history:
        messages += chat_history
    messages.append(("user", user_query))
    prompt_template = ChatPromptTemplate.from_messages(messages)
    chain = prompt_template | llm | StrOutputParser()
    return chain.invoke({"input": user_query})


c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from crewai import Agent, Task, Crew, Process, LLM
# =========================
# LLM p/ CrewAI
# =========================
CREW_LLM = LLM(
    model=CREW_MODEL,            # "ollama/phi4"  <- importante o prefixo do provedor
    base_url=OLLAMA_BASE_URL,    # ex.: "http://localhost:11434"
    temperature=LLM_TEMPERATURE,
)

### Especialista em informações da cidade

In [5]:
from crewai import Agent
from textwrap import dedent

city_info_agent = Agent(
    role="Especialista em informações da cidade",
    goal=dedent(
        """Reunir informações úteis sobre o destino escolhido, com base nos interesses e datas da viagem,
        ajudando viajantes a entender o contexto geral do local."""
    ),
    backstory=dedent(
        """
        Sou um especialista com amplo conhecimento sobre cidades ao redor do mundo,
        capaz de fornecer detalhes atualizados sobre clima, eventos culturais, segurança,
        costumes locais e outros aspectos práticos para quem pretende visitar o local.
        Sou apaixonado por compartilhar as melhores experiências e 'joias escondidas' do local.
        """
    ),
    llm=CREW_LLM,
    tools=[SearchTools.search_duckduckgo],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

### Logística e informações da cidade

In [6]:
logistics_expert_agent = Agent(
    role="Especialista em logística de viagem",
    goal="Identificar as melhores opções logísticas para a viagem, com foco em praticidade, custo-benefício e conforto.",
    backstory=dedent(
        """
        Sou um profissional focado em planejar a logística da viagem: transporte, hospedagem e deslocamento local.
        Tenho conhecimento sobre companhias aéreas, apps de mobilidade, regiões seguras para se hospedar e otimização de trajetos.
        """
    ),
    llm=CREW_LLM,
    tools=[
        SearchTools.search_duckduckgo,
        CalculatorTools.calculate
    ],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

### Especialista em criação de itinerário

In [7]:
itinerary_planner_agent = Agent(
    role="Planejador de itinerário personalizado",
    goal="Criar um roteiro completo com base nas preferências do usuário.",
    backstory=dedent(
        """
        Sou um guia profissional apaixonado por viagens, especialista em organização de itinerários personalizados.
        Minha missão é integrar dados sobre clima, atrações, eventos e logística em uma experiência otimizada para o turista.
        """
    ),
    llm=CREW_LLM,
    tools=[SearchTools.search_duckduckgo],
    verbose=True,
    max_iter=10,
    allow_delegation=False,
)

In [8]:
language_guide_agent = Agent(
    role="Especialista em comunicação e etiqueta local",
    goal="Gerar um guia traduzido com frases úteis, dicas de etiqueta e expressões práticas com base nas atividades do roteiro.",
    backstory=dedent(
        """
        Sou um especialista em línguas e culturas do mundo.
        Meu trabalho é ajudar turistas a se comunicarem melhor no destino, traduzindo expressões essenciais relacionadas ao roteiro,
        como pedidos em restaurantes, orientações para transporte e interações cotidianas.
        Também forneço dicas culturais para evitar gafes e tornar a experiência mais fluida e respeitosa.
        """
    ),
    llm=CREW_LLM,
    tools=[SearchTools.search_duckduckgo],
    verbose=True,
    max_iter=5,
    allow_delegation=False,
)

## Criação das Tarefas (Tasks)

In [9]:
from_city = "Natal" 
destination_city = "Argentina" 
date_from = "23 de novembro de 2025" 
date_to = "30 de novembro de 2025" 
interests = "arte, tecnologia, vinhos, arquitetura" 

print(f"""
Viajando de: {from_city}
Para: {destination_city}
Chegada (início da viagem): {date_from}
Partida (fim da viagem): {date_to}
""")


Viajando de: Natal
Para: Argentina
Chegada (início da viagem): 23 de novembro de 2025
Partida (fim da viagem): 30 de novembro de 2025



### Task: Coleta informações da cidade

In [10]:
from crewai import Task

def city_info_task(agent, from_city, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes locais.
            Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e quaisquer atividades que se alinhem às preferências do usuário.
            Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.
            Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.

            Viajando de: {from_city}
            Para: {destination_city}
            Interesses do viajante: {interests}
            Chegada: {date_from}
            Partida: {date_to}

            Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.
            """
        ),
        expected_output=dedent(
            f"""
            Um guia detalhado (em formato markdown) em português que inclui:
            - Resumo da cidade e sua cultura;
            - Uma lista selecionada de lugares recomendados para visitar e eventos (se houver);
            - Um detalhamento das despesas diárias, como custos médios com alimentação;
            - Recomendações de segurança;
            - Dicas de costumes locais;
            """
        ),
        agent=agent,
        output_file='relatorio_local.md',
    )

### Task: Planejamento de logística

In [11]:
def plan_logistics_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Planejar a logística da viagem.
            Identificar as melhores opções de hospedagem, voos e transporte local considerando:

            Destino: {destination_city}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            Interesses: {interests}

            Avalie preço, localização, conveniência e segurança.
            """
        ),
        expected_output=dedent(
            f"""
            Relatório em português (em formato markdown):
            - Sugestão de hospedagens, preferencialmente com localização estratégica;
            - Opções de voos ou meios de transporte para chegada/partida;
            - Sugestões de deslocamento dentro da cidade;
            - Estimativas de custo para cada item;
            """
        ),
        context=context,
        agent=agent,
        output_file='relatorio_logistica.md',
    )

### Task: Construção itinerário final

In [12]:
def build_itinerary_task(context, agent, destination_city, interests, date_from, date_to):
    return Task(
        description=dedent(
            f"""
            Esta tarefa sintetiza todas as informações para criar o roteiro final da viagem.
            Com base nas informações dos outros agentes, desenvolva um itinerário detalhado.
            Cada dia deve conter atividades, clima, transporte, refeições e estimativa de gastos.

            Destino: {destination_city}
            Interesses: {interests}
            Data de chegada: {date_from}
            Data de partida: {date_to}
            """
        ),
        expected_output=dedent("""
        Documento em português que inclui (em formato markdown):
        - Boas vindas e apresentação da cidade em até 2 parágrafos.
        - Programação diária com sugestões de horários e atividades
        - Atrações distribuídas por região e logística
        - Clima previsto, sugestões de transporte e alimentação
        - Eventos e festivais sazonais (se houver)
        - Estimativa de gastos por dia, custo médio das despesas diárias e pontos turísticos.
        - Visão geral dos destaques da cidade com base nas recomendações do guia.
        - Outras dicas e informações adicionais para uma viagem e estadia tranquila.
        """),
        context=context,
        agent=agent,
        output_file='roteiro_viagem.md',
    )

In [13]:
def language_guide_task(context, agent, destination_city):
    return Task(
        description=dedent(
            f"""
            Com base no destino ({destination_city}) e nas atividades previstas no roteiro, monte um guia traduzido com frases úteis e dicas culturais.
            Deve ser no idioma falado nesse local (ou também em inglês, caso seja uma língua aceitável para ser usada por turistas nesse local).

            Inclua expressões comuns para situações que podem ser úteis, como:
            - Interações em restaurantes (como fazer pedidos, pagar a conta, pedir recomendações);
            - Deslocamento (perguntar por rotas, chamar transporte, entender sinalização);
            - Situações cotidianas (compras, pedidos de ajuda, saudações e agradecimentos);
            - Dicas de etiqueta local que o turista deve saber (gestos, hábitos, regras sociais).

            Use linguagem clara e educativa.

            Considere o seguinte roteiro de viagem:
            '{context}'
            """
        ),
        expected_output=dedent(
            """
            Um mini-guia em português contendo:
            - Lista de frases traduzidas organizadas por situação (restaurantes, transporte, compras, etc.);
            - Dicas de etiqueta e costumes locais;
            - Sugestões de como pronunciar corretamente (quando aplicável);
            - Recomendações práticas para comunicação eficaz no destino.
            """
        ),
        agent=agent,
        output_file='guia_comunicacao.md',
    )

### Executando a criação das tasks

In [14]:
city_info = city_info_task(
  city_info_agent,
  from_city,
  destination_city,
  interests,
  date_from,
  date_to
)

In [15]:
plan_logistics = plan_logistics_task(
  [city_info],
  logistics_expert_agent,
  destination_city,
  interests,
  date_from,
  date_to
)

In [16]:
build_itinerary = build_itinerary_task(
  [city_info, plan_logistics],
  itinerary_planner_agent,
  destination_city,
  interests,
  date_from,
  date_to,
)

In [17]:
language_guide = language_guide_task(
  [build_itinerary],
  language_guide_agent,
  destination_city,
)

## Criação da equipe (Crew)

In [18]:
from crewai import Crew, Process

# crew = Crew(
#   agents=[city_info_agent, logistics_expert_agent, itinerary_planner_agent],
#   tasks=[city_info, plan_logistics, build_itinerary],
#   process=Process.sequential,
#   full_output=True,
#   share_crew=False,
#   verbose=True,
#   max_rpm=15
# )
# result = crew.kickoff()

from crewai import Crew, Process
crew = Crew(
  agents=[city_info_agent, logistics_expert_agent, itinerary_planner_agent, language_guide_agent],
  tasks=[city_info, plan_logistics, build_itinerary, language_guide],
  process=Process.sequential,
  full_output=True,
  share_crew=False,
  verbose=True,
  max_rpm=15
)
result = crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cf3c4304-d06e-4f64-88d6-36df5d1436e8                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Especialista em informações da cidade                                                                   │
│                                                                                                                 │
│  Task:                                                                                                          │
│  Levantar informações detalhadas sobre a cidade e coletar dados úteis sobre clima, segurança e costumes         │
│  locais.                                                                                                        │
│  Identificar marcos culturais, pontos históricos, locais de entretenimento, experiências gastronômicas e        │
│  quaisquer atividades que se alinhem às preferências do usuário.                                                │
│  Também destacar eventos e festivais sazonais que podem ser de interesse durante a visita do viajante.          │
│  Use as ferramentas disponíveis para buscar fontes atualizadas e confiáveis.                                    │
│                                                                                                                 │
│  Viajando de: Natal                                                                                             │
│  Para: Argentina                                                                                                │
│  Interesses do viajante: arte, tecnologia, vinhos, arquitetura                                                  │
│  Chegada: 23 de novembro de 2025                                                                                │
│  Partida: 30 de novembro de 2025                                                                                │
│                                                                                                                 │
│  Seja criterioso, como se estivesse ajudando alguém a ter uma experiência inesquecível.                         │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

c:\Users\niels\anaconda3\envs\GPU_Tensor\lib\site-packages\rich\live.py:231: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[32;1m[1;3msnippet: 19 de nov. de 2024 — Argentina, oficialmente República Argentina (pronunciado em espanhol: ), é
o segundo maior país da América do Sul em território e o ..., title: Argentina – Wikipédia, a enciclopédia livre, 
link: https://pt.wikipedia.org/wiki/Argentina, snippet: há 3 dias — O Brasil conquistou nove medalhas na Sessão de 
Vinhos Espumantes do Concurso Mundial de Bruxelas, realizado no começo de setembro em Chisinau, ..., title: Brasil 
tem 9 vinhos espumantes premiados em competição ..., link: 
https://saleteemsociedade.com/brasil-tem-9-vinhos-espumantes-premiados-em-competicao-internacional/, snippet: 31 de
jan. de 2025 — eventos que refletem a rica cultura local . Através do programa, artistas locais têm encontrado uma 
plataforma para divulgar seu trabalho, e a comunidade tem ... 447 páginas, title: A GESTÃO DA CULTURA CARIOCA, 
link: https://cultura.prefeitura.rio/wp-content/uploads/sites/38/2025/01/bookatualizado_2025-1_compressed.pdf, 
snippet: 29 de nov. de 2024 — É com imensa satisfação que apresentamos a segunda edição da. Revista Memorial 
Cultural, fruto da colaboração inédita e enrique- cedora entre a Fundação ... 51 páginas, title: Cusco respira arte
De O Eternauta a Superpunk Além da dor, link: 
https://memorial.org.br/wp-content/uploads/2024/11/Memorial-Cultural-2.pdf[0m
🚀 Crew: crew
└── 📋 Task: a062394b-9c4d-4f83-b500-585b997e74d8
    Status: Executing Task...
    └── 🔧 Used Pesquisa na internet com DuckDuckGo (1)

...

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"query\": \"cultural etiquette in Argentina\"}"                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  snippet: 1 week ago - The culture of Argentina is as ... European immigration (mainly Italian and Spanish),    │
│  so it is strongly linked to the Western world and its Catholic religious tradition ...., title: Wikipedia      │
│  Culture of Argentina - Wikipedia, link: https://en.wikipedia.org/wiki/Culture_of_Argentina, snippet: May 6,    │
│  2025 - Immerse yourself in Argentina ' s culture and traditions with this guide. Get insight into the          │
│  country's food, dance, and customs before your trip., title: Celebrity Current Argentina Culture Guide: 11     │
│  Can’t-Miss Customs & Traditions | Celebrity Cruises, link:                                                     │
│  https://www.celebritycruises.com/blog/argentina-culture-guide, snippet: Argentina is a diverse and culturally  │
│  rich country located in South America. Known for its stunning landscapes, vibrant cities, and deep-rooted      │
│  traditions, it is a popular destination for travellers seeking adventure, history, and cultural experiences.   │
│  This guide covers key aspects of Argentina , including the type of Spanish spoken, etiquette , and customs.    │
│  The Spanish Spoken in Argentina The ..., title: Argentina: language, customs and etiquette — Learn Spanish,    │
│  link: https://www.learn-spanish.uk/articles/argentina-language-customs-and-etiquette, snippet: Argentina is a  │
│  country of contrasts and colors, where European influence blends with indigenous roots, and urban rhythms      │
│  coexist with rural traditions. To truly understand and appreciate Argentina , one must look beyond the         │
│  surface of its famous landmarks and dive into its customs, social habits, and cultural rituals that shape      │
│  everyday life., title: Argentinian Customs and Traditions | Know Argentina, link:                              │
│  https://knowargentina.com/argentinian-customs-and-traditions                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Especialista em comunicação e etiqueta local                                                            │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Mini-Guia para Turistas em Buenos Aires                                                                      │
│                                                                                                                 │
│  ## Frases Úteis Organizadas por Situação                                                                       │
│                                                                                                                 │
│  ### Restaurantes                                                                                               │
│  - **Pedir uma mesa**: "Una mesa para dos, por favor." (A table for two, please.)                               │
│    - Pronúncia: [oo-nah mes-tah proh-doh vos]                                                                   │
│  - **Fazer um pedido**: "Me gustaría pedir el menú del día." (I would like to order the daily menu.)            │
│    - Pronúncia: [meh goo-stah-reee puh-deer el meh-noo dehl dee-ah]                                             │
│  - **Pagar a conta**: "La cuenta, por favor." (The bill, please.)                                               │
│    - Pronúncia: [lah kwen-tah proh-fah-vor]                                                                     │
│  - **Pedir recomendações**: "¿Qué nos recomienda?" (What do you recommend?)                                     │
│    - Pronúncia: [keh nohs kooh-mee-en-dee-ah]                                                                   │
│                                                                                                                 │
│  ### Transporte                                                                                                 │
│  - **Perguntar por rotas**: "¿Cómo llego a la Plaza de Mayo?" (How do I get to Plaza de Mayo?)                  │
│    - Pronúncia: [koh-moh yeh-goh ah lah plah-zah deh mah-yoh]                                                   │
│  - **Chamar transporte público**: "Un taxi, por favor." (A taxi, please.)                                       │
│    - Pronúncia: [oon tah-hee-seh proh-fah-vor]                                                                  │
│                                                                                                                 │
│  ### Compras                                                                                                    │
│  - **Negociar preço**: "¿Cuánto cuesta esto?" (How much does this cost?)                                        │
│    - Pronúncia: [kwan-toh kwehs-tah eh-stoh]                                                                    │
│  - **Pedir ajuda**: "¿Puede ayudarme, por favor?" (Can you help me, please?)                                    │
│    - Pronúncia: [poo-deh ah-yoo-dar-meh proh-fah-vor]                                                           │
│                                                                                                                 │
│  ### Situações Cotidianas                                                                                       │
│  - **Saudações**:                                                                                               │
│    - Bom dia: "Buenos días." (Good morning.)                                                                    │
│      - Pronúncia: [bwehn-os dee-as]                    

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: cc702c0d-ee5c-4000-9699-b7f8966394fd                                                                     │
│  Agent: Especialista em comunicação e etiqueta local                                                            │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cf3c4304-d06e-4f64-88d6-36df5d1436e8                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Mini-Guia para Turistas em Buenos Aires                                                        │
│                                                                                                                 │
│  ## Frases Úteis Organizadas por Situação                                                                       │
│                                                                                                                 │
│  ### Restaurantes                                                                                               │
│  - **Pedir uma mesa**: "Una mesa para dos, por favor." (A table for two, please.)                               │
│    - Pronúncia: [oo-nah mes-tah proh-doh vos]                                                                   │
│  - **Fazer um pedido**: "Me gustaría pedir el menú del día." (I would like to order the daily menu.)            │
│    - Pronúncia: [meh goo-stah-reee puh-deer el meh-noo dehl dee-ah]                                             │
│  - **Pagar a conta**: "La cuenta, por favor." (The bill, please.)                                               │
│    - Pronúncia: [lah kwen-tah proh-fah-vor]                                                                     │
│  - **Pedir recomendações**: "¿Qué nos recomienda?" (What do you recommend?)                                     │
│    - Pronúncia: [keh nohs kooh-mee-en-dee-ah]                                                                   │
│                                                                                                                 │
│  ### Transporte                                                                                                 │
│  - **Perguntar por rotas**: "¿Cómo llego a la Plaza de Mayo?" (How do I get to Plaza de Mayo?)                  │
│    - Pronúncia: [koh-moh yeh-goh ah lah plah-zah deh mah-yoh]                                                   │
│  - **Chamar transporte público**: "Un taxi, por favor." (A taxi, please.)                                       │
│    - Pronúncia: [oon tah-hee-seh proh-fah-vor]                                                                  │
│                                                                                                                 │
│  ### Compras                                                                                                    │
│  - **Negociar preço**: "¿Cuánto cuesta esto?" (How much does this cost?)                                        │
│    - Pronúncia: [kwan-toh kwehs-tah eh-stoh]                                                                    │
│  - **Pedir ajuda**: "¿Puede ayudarme, por favor?" (Can you help me, please?)                                    │
│    - Pronúncia: [poo-deh ah-yoo-dar-meh proh-fah-vor]                                                           │
│                                                                                                                 │
│  ### Situações Cotidianas                                                                                       │
│  - **Saudações**:                                                                                               │
│    - Bom dia: "Buenos días." (Good morning.)          

## Visualizar resultado

In [19]:
from IPython.display import display, Markdown

def load_markdown(file_path):
  try:
    with open(file_path, 'r', encoding='utf-8') as file:
      content = file.read()
      content = content.replace("```markdown", "").replace("```", "")
      return content
  except Exception as e:
    print(f"Erro ao carregar o arquivo: {str(e)}")
    return None

In [20]:
def display_markdown(content):
  return display(Markdown(content))

In [21]:
md = load_markdown("roteiro_viagem.md")
display_markdown(md)

The detailed itinerary for your trip to Buenos Aires from November 23 to November 30, 2025, is provided above. It includes daily activities, climate expectations, transportation options, meal suggestions, and estimated costs based on your interests in art, technology, wines, and architecture. Enjoy your cultural exploration of this vibrant city!

## Converter em PDF

In [22]:
import markdown2
from weasyprint import HTML

def convert_md_to_pdf(file_md, file_pdf):
  text_md = load_markdown(file_md)
  if text_md is not None:
    html_body = markdown2.markdown(text_md)
    style_css = """
    <style>
        body {
            font-family: Arial, Helvetica, sans-serif;
            font-size: 12pt;
            line-height: 1.5;
        }
    </style>
    """
    full_html = f"<html><head>{style_css}</head>{html_body}</html>"
    HTML(string=full_html).write_pdf(file_pdf)

In [27]:
convert_md_to_pdf('relatorio_local.md', 'relatorio_local.pdf')